In [0]:
import requests
import json
import pandas as pd
import numpy as np
import time
from collections import defaultdict
import pickle
from pathlib import Path
import dask.dataframe as dd

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
ROOT = '/content/gdrive/My Drive/ML/stuff'
DATA = f'{ROOT}/data/'
OUT = f'{ROOT}/out'

if not Path(OUT).exists():
    ! mkdir "$OUT"

In [0]:

url = "http://e621.net"
username = ""
project_name = "RandomStats/1.0"
headers = {
    'User-Agent': f"{project_name} {username}"
}
keys = ["rating", "author", "id", "sample_url", "score", "fav_count", "tags", "file_url"]

rows = []


def clean(x):
    if type(x) == int:
        return x
    return x.encode('ascii', 'ignore').strip()


first_id = 1872005

In [0]:
for i, u in enumerate(users):
    id = first_id
    print(u)
    while id > 0:
        post_url = url + f"/post/index.json?before_id={id}&limit=320&tags=fav:{u}"
        while True:
            time.sleep(1)
            try:
                r = requests.get(post_url, headers=headers)
                if r.status_code == 200:
                    break
            except Exception as e:
                print(e)
        str = r.content
        content = json.loads(str)
        for post in content:
            entry = {key: clean(post[key]) for key in keys}
            id = entry["id"]
            users[u].add(id)
        print(id)
        if len(content) < 320:
            break
    print(f"{i} / {len(users)}")

In [0]:
users2 = {}
for u in users:
    if len(users[u]) > 100:
        users2[u] = users[u]
len(users2)
users = users2
len(users)

In [0]:

while id > 1000:
    post_url = url + "/post/index.json?before_id={}&limit=320".format(id)
    while True:
        time.sleep(1)
        try:
            r = requests.get(post_url, headers=headers)
            if r.status_code == 200:
                break
        except Exception as e:
            print(e)
    str = r.content
    content = json.loads(str)
    for post in content:
        entry = {key: clean(post[key]) for key in keys}
        rows.append(entry)
        id = entry["id"]


In [0]:
df = pd.DataFrame(rows)
df.to_csv("out.csv")

In [0]:
df = pd.read_csv(f"{DATA}/data.csv")
len(df)

In [0]:
users = defaultdict(lambda: [])
index = 0

In [0]:
index = 0

In [0]:

while index < len(ids):
    id = ids[index]
    post_url = url + "/favorite/list_users.json?id={}".format(id)
    while True:
        time.sleep(1)
        try:
            r = requests.get(post_url, headers=headers)
            if r.status_code == 200:
                break
        except Exception as e:
            print(e)
    str = r.content
    content = json.loads(str)
    for user in content["favorited_users"].split(","):
        users[user].append(id)
    index += 1



In [0]:
pickle.dump(dict(users), open(f"{DATA}/users.p", "wb" ) )

In [0]:
users = pickle.load(open(f"{DATA}/users.p", "rb"))

In [0]:
df["tags_split"] = df["tags"].apply(lambda x: set(x.split()))
df = df.drop(columns=["tags"])